In [45]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import BaggingRegressor, BaggingClassifier 
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier 
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, accuracy_score, classification_report
from sklearn.metrics import mean_squared_error

Exercice sur le Random Forest avec Bagging/Pasting
Objectif :

    Mettre en place un modèle de Random Forest pour prédire le prix (target de régression) des vins.
    Mettre en place un modèle de Random Forest pour classifier les vins en fonction de leur variété.

Étapes :

    Nettoyage des données :
        Vérifiez s'il y a des valeurs manquantes dans le dataset et traitez-les.
        Convertissez les variables catégorielles en variables numériques si nécessaire.
        Séparez le dataset en features (X) et la target variable (y) pour la régression et la classification.

    Random Forest pour la Régression (Prédiction de Prix) :
        Utilisez la bibliothèque scikit-learn en Python.
        Divisez le dataset en ensembles d'entraînement (training set) et de test (test set).
        Entraînez un modèle de Random Forest pour prédire le prix des vins.
        Évaluez les performances du modèle en utilisant des métriques appropriées pour la régression (par exemple, MSE, RMSE).

    Random Forest pour la Classification (Variété de Vin) :
        Convertissez la variable de variété en une variable numérique si nécessaire.
        Divisez le dataset en ensembles d'entraînement (training set) et de test (test set).
        Entraînez un modèle de Random Forest pour classifier la variété de vin.
        Évaluez les performances du modèle en utilisant des métriques appropriées pour la classification (par exemple, précision, rappel, F1-score).

    Bagging ou Pasting :
        Comparez les performances du Random Forest avec bagging et du Random Forest avec pasting.
        Utilisez différentes valeurs pour le nombre d'estimateurs (arbres) et observez comment cela affecte les performances.
        Choisissez une métrique d'évaluation appropriée pour comparer les deux approches.

    Analyse des Résultats :
        Analysez et interprétez les résultats obtenus.
        Identifiez les hyperparamètres optimaux pour chaque modèle.

In [46]:
# Chargement du dataset
url = "datasets/wine.csv" 
data = pd.read_csv(url)

# Nettoyage des données
# Remplacer les valeurs numériques par la moyenne
numeric_columns = data.select_dtypes(include='number').columns
data[numeric_columns] = data[numeric_columns].fillna(data[numeric_columns].mean())

# Remplacer les valeurs catégorielles par la mode des autres catégories
categorical_columns = data.select_dtypes(exclude='number').columns
for col in categorical_columns:
    data[col] = data.groupby(col)[col].transform(lambda x: x.fillna(x.mode()[0]))

# Séparation du dataset en features (X) et target variable (y) pour la régression
X_reg = data.drop(['price'], axis=1)  # Features pour la régression
y_reg = data['price']  # Target variable pour la régression

# Calcul des corrélations entre les colonnes catégorielles et la variable cible
correlations = X_reg[categorical_columns].corrwith(y_reg)

# Sélection des colonnes catégorielles avec une corrélation significative (par exemple, supérieure à 0.1)
significant_categorical_columns = correlations[abs(correlations) > 0.1].index.tolist()

# Divisez le dataset en ensembles d'entraînement et de test pour la régression
X_reg_train, X_reg_test, y_reg_train, y_reg_test = train_test_split(X_reg, y_reg, test_size=0.2, random_state=42)

# Création d'une pipeline
regression_pipeline = Pipeline([
    # Prétraitement des données
    ('preprocessor', ColumnTransformer(transformers=[
        ('num', SimpleImputer(strategy='mean'), numeric_columns),
        ('cat', OneHotEncoder(), categorical_columns)
    ], remainder='passthrough')),
    # Modèle de régression
    ('regressor', RandomForestRegressor(n_estimators=100, max_features='sqrt', max_depth=10, random_state=42))
])

# Entraînement de la pipeline
regression_pipeline.fit(X_reg_train, y_reg_train)

# Prédictions sur l'ensemble de test pour la régression
y_reg_pred = regression_pipeline.predict(X_reg_test)

# Évaluation des performances pour la régression
mse = mean_squared_error(y_reg_test, y_reg_pred)
rmse = mse**0.5
print(f"RMSE (Régression) : {rmse}")

NameError: name 'SimpleImputer' is not defined

In [ ]:
# Chargement du dataset
url = "datasets/wine.csv" 
data = pd.read_csv(url)

In [ ]:
# Calcul du ratio entre les valeurs manquantes et le total des valeurs par colonne
ratio_missing_values = data.isna().sum() / data.count()

# Affichage du résultat
print(ratio_missing_values)

Unnamed: 0               0.000000
country                  0.000485
description              0.000000
designation              0.405001
points                   0.000000
price                    0.074362
province                 0.000485
region_1                 0.195421
region_2                 1.573123
taster_name              0.253010
taster_twitter_handle    0.316055
title                    0.000000
variety                  0.000008
winery                   0.000000
dtype: float64


In [ ]:
# Remplacement des valeurs numériques par la moyenne
numeric_columns = data.select_dtypes(include='number').columns
data[numeric_columns] = data[numeric_columns].fillna(data[numeric_columns].mean())


In [ ]:
categorical_columns = data.select_dtypes(exclude='number').columns
for col in categorical_columns:
    data[col] = data.groupby(col)[col].transform(lambda x: x.fillna(x.mode()[0]))

In [ ]:
data.head()

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,35.363389,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.000000,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.000000,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.000000,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.000000,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks


In [ ]:
ratio_missing_values = data.isna().sum() / data.count()

# Affichage du résultat
print(ratio_missing_values)

Unnamed: 0               0.000000
country                  0.000485
description              0.000000
designation              0.405001
points                   0.000000
price                    0.000000
province                 0.000485
region_1                 0.195421
region_2                 1.573123
taster_name              0.253010
taster_twitter_handle    0.316055
title                    0.000000
variety                  0.000008
winery                   0.000000
dtype: float64
